# <img src="../assets/favicon.png" alt="Logo" width="50"/> **CHardy Tecno Store**

---

# ⭐ Contrucción del Modelo Estrella 

## 1. Carga de los DataFrame ya Transformados y Limpios

In [1]:
import pandas as pd
import numpy as np
import os 

PROCESSED_DATA_PATH = 'data/processed'

def load_parquet_file(filename):
    """Carga un archivo Parquet desde la ruta predefinida."""
    full_path = os.path.join(PROCESSED_DATA_PATH, filename)
    try:
        df = pd.read_parquet(full_path)
        print(f"✔️ Cargado: {filename}")
        return df
    except FileNotFoundError:
        print(f"❌ Error: Archivo no encontrado en la ruta {full_path}. Asegúrate de que los archivos Parquet existen.")
        return None

# Cargar las tablas ETL ya limpias y transformadas
df_ventas = load_parquet_file('fact_ventas.parquet')
df_fecha = load_parquet_file('dim_calendario.parquet')
df_productos = load_parquet_file('dim_productos.parquet')
df_clientes = load_parquet_file('dim_clientes.parquet')
df_sucursales = load_parquet_file('dim_sucursales.parquet') 
df_canales = load_parquet_file('dim_canales.parquet')

# Verificar si todas las tablas se cargaron correctamente
if any(df is None for df in [df_ventas, df_fecha, df_productos, df_clientes, df_sucursales, df_canales]):
    print("\nDeteniendo el proceso. Corrige las rutas o nombres de archivo Parquet.")
    exit()

✔️ Cargado: fact_ventas.parquet
✔️ Cargado: dim_calendario.parquet
✔️ Cargado: dim_productos.parquet
✔️ Cargado: dim_clientes.parquet
✔️ Cargado: dim_sucursales.parquet
✔️ Cargado: dim_canales.parquet


## 2. Conversión de Tipos (Asegurar Claves de Unión)

In [2]:
print("\n--- Inspección Detallada de df_ventas (.info()) ---")
df_ventas.info()


--- Inspección Detallada de df_ventas (.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838945 entries, 0 to 838944
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   transaction_id               838945 non-null  int64         
 1   fecha                        838945 non-null  datetime64[ns]
 2   año                          838945 non-null  int64         
 3   mes                          838945 non-null  int64         
 4   cliente_id                   838945 non-null  int64         
 5   producto_id                  838945 non-null  int64         
 6   canal_id                     838945 non-null  int64         
 7   sucursal_id                  838945 non-null  int64         
 8   cantidad                     838945 non-null  int64         
 9   precio_unitario_ars_nominal  838945 non-null  float64       
 10  monto_venta_ars_nominal      838945 non

In [3]:
# Es buena práctica asegurar que las claves de unión tengan el tipo de dato correcto
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])
df_ventas['sucursal_id'] = df_ventas['sucursal_id'].astype('Int64')
df_fecha['fecha'] = pd.to_datetime(df_fecha['fecha'])

# Asegurar que las claves ID sean tipo entero para uniones eficientes
id_cols = ['producto_id', 'cliente_id', 'canal_id', 'sucursal_id']
for col in id_cols:
    df_ventas[col] = df_ventas[col].astype('Int64')
    if col in df_productos.columns: df_productos[col] = df_productos[col].astype('Int64')
    if col in df_clientes.columns: df_clientes[col] = df_clientes[col].astype('Int64')
    if col in df_canales.columns: df_canales[col] = df_canales[col].astype('Int64')
    if col in df_sucursales.columns: df_sucursales[col] = df_sucursales[col].astype('Int64')

# Muestra los tipos de datos de las columnas clave en la tabla de hechos
print("--- Tipos de Datos en df_ventas ---")
print(df_ventas[['fecha', 'producto_id', 'cliente_id', 'canal_id', 'sucursal_id']].dtypes)

# Muestra los tipos de datos en las dimensiones
print("\n--- Tipos de Datos en df_calendario (Clave) ---")
print(df_fecha[['fecha']].dtypes)

print("\n--- Tipos de Datos en df_productos (Clave) ---")
print(df_productos[['producto_id']].dtypes)

print("\n--- Tipos de Datos en df_clientes (Clave) ---")
print(df_clientes[['cliente_id']].dtypes)

print("\n--- Tipos de Datos en df_canales (Clave) ---")
print(df_canales[['canal_id']].dtypes)

print("\n--- Tipos de Datos en df_sucursales (Clave) ---")
print(df_sucursales[['sucursal_id']].dtypes)

--- Tipos de Datos en df_ventas ---
fecha          datetime64[ns]
producto_id             Int64
cliente_id              Int64
canal_id                Int64
sucursal_id             Int64
dtype: object

--- Tipos de Datos en df_calendario (Clave) ---
fecha    datetime64[ns]
dtype: object

--- Tipos de Datos en df_productos (Clave) ---
producto_id    Int64
dtype: object

--- Tipos de Datos en df_clientes (Clave) ---
cliente_id    Int64
dtype: object

--- Tipos de Datos en df_canales (Clave) ---
canal_id    Int64
dtype: object

--- Tipos de Datos en df_sucursales (Clave) ---
sucursal_id    Int64
dtype: object


In [4]:
# Reviso cada uno de los df
print("\n--- Inspección Detallada de df (.info()) ---")
df_clientes.info()


--- Inspección Detallada de df (.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cliente_id      15000 non-null  Int64 
 1   nombre_cliente  15000 non-null  object
 2   email_cliente   15000 non-null  object
 3   ciudad_cliente  15000 non-null  object
 4   segmento_base   15000 non-null  object
dtypes: Int64(1), object(4)
memory usage: 600.7+ KB


In [6]:
# Funcion para chequear la consistencias de Claves para mergear
def check_key_consistency(df_left, key_left, df_right, key_right):
    """Verifica si los tipos de datos de las claves de unión coinciden."""
    type_left = df_left[key_left].dtype
    type_right = df_right[key_right].dtype
    
    if type_left == type_right:
        print(f"✔️ Éxito: Las claves '{key_left}' y '{key_right}' coinciden. Tipo: {type_left}")
    else:
        print(f"❌ Advertencia: Tipos de claves no coinciden. {key_left}: {type_left} vs. {key_right}: {type_right}")

# Chequeo de Claves
check_key_consistency(df_ventas, 'fecha', df_fecha, 'fecha')
check_key_consistency(df_ventas, 'producto_id', df_productos, 'producto_id')
check_key_consistency(df_ventas, 'cliente_id', df_clientes, 'cliente_id')
check_key_consistency(df_ventas, 'canal_id', df_canales, 'canal_id')
check_key_consistency(df_ventas, 'sucursal_id', df_sucursales, 'sucursal_id')

✔️ Éxito: Las claves 'fecha' y 'fecha' coinciden. Tipo: datetime64[ns]
✔️ Éxito: Las claves 'producto_id' y 'producto_id' coinciden. Tipo: Int64
✔️ Éxito: Las claves 'cliente_id' y 'cliente_id' coinciden. Tipo: Int64
✔️ Éxito: Las claves 'canal_id' y 'canal_id' coinciden. Tipo: Int64
✔️ Éxito: Las claves 'sucursal_id' y 'sucursal_id' coinciden. Tipo: Int64


## 3. Auditoria en bloques para iniciar Uniones/Joins

In [7]:
# Funcion para auditoria
def auditar_join(df_hechos, df_dim, clave):
    ids_hechos = set(df_hechos[clave].dropna())
    ids_dim = set(df_dim[clave].dropna())
    faltantes = ids_hechos - ids_dim
    print(f"[{clave}] Total hechos: {len(ids_hechos)} | Total dim: {len(ids_dim)} | Sin match: {len(faltantes)}")
    return faltantes

# Auditoría en bloque
faltantes_clientes   = auditar_join(df_ventas, df_clientes,   'cliente_id')
faltantes_productos  = auditar_join(df_ventas, df_productos,  'producto_id')
faltantes_canales    = auditar_join(df_ventas, df_canales,    'canal_id')
faltantes_sucursales = auditar_join(df_ventas, df_sucursales, 'sucursal_id')
faltantes_fechas     = auditar_join(df_ventas, df_fecha, 'fecha')

# Merge con flags de validez
df_fact = (
    df_ventas
    .merge(df_clientes,   on='cliente_id',   how='left', indicator=True)
    .rename(columns={'_merge':'merge_clientes'})
    .merge(df_productos,  on='producto_id',  how='left', indicator=True)
    .rename(columns={'_merge':'merge_productos'})
    .merge(df_canales,    on='canal_id',     how='left', indicator=True)
    .rename(columns={'_merge':'merge_canales'})
    .merge(df_sucursales, on='sucursal_id',  how='left', indicator=True)
    .rename(columns={'_merge':'merge_sucursales'})
    .merge(df_fecha, on='fecha',        how='left', indicator=True)
    .rename(columns={'_merge':'merge_calendario'})
)

# Flags de validez
df_fact['es_cliente_valido']   = (df_fact['merge_clientes']   == 'both').astype(int)
df_fact['es_producto_valido']  = (df_fact['merge_productos']  == 'both').astype(int)
df_fact['es_canal_valido']     = (df_fact['merge_canales']    == 'both').astype(int)
df_fact['es_sucursal_valida']  = (df_fact['merge_sucursales'] == 'both').astype(int)
df_fact['es_fecha_valida']     = (df_fact['merge_calendario'] == 'both').astype(int)

# Auditoría final
print("Clientes sin match:",   (df_fact['es_cliente_valido']   == 0).sum())
print("Productos sin match:",  (df_fact['es_producto_valido']  == 0).sum())
print("Canales sin match:",    (df_fact['es_canal_valido']     == 0).sum())
print("Sucursales sin match:", (df_fact['es_sucursal_valida']  == 0).sum())
print("Fechas sin match:",     (df_fact['es_fecha_valida']     == 0).sum())


[cliente_id] Total hechos: 15000 | Total dim: 15000 | Sin match: 0
[producto_id] Total hechos: 28 | Total dim: 28 | Sin match: 0
[canal_id] Total hechos: 2 | Total dim: 2 | Sin match: 0
[sucursal_id] Total hechos: 26 | Total dim: 26 | Sin match: 0
[fecha] Total hechos: 2557 | Total dim: 2557 | Sin match: 0
Clientes sin match: 0
Productos sin match: 0
Canales sin match: 0
Sucursales sin match: 0
Fechas sin match: 0


## 4. Construcción del Modelo Estrella (Uniones/Joins)

In [9]:
# libero memoria del df usado en auditoria.
del df_fact


print("\n⭐ Iniciando la construcción del Modelo Estrella...")
# Partimos de la tabla de Hechos (df_ventas)
df_modelo_estrella = df_ventas.copy()


⭐ Iniciando la construcción del Modelo Estrella...


### 4.1. Unir a Dimensión Fecha

In [10]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_fecha.drop(columns=['año', 'mes', 'día'], errors='ignore'), # Evitar duplicar columnas si ya se hicieron las uniones antes
    on='fecha',
    how='left'
)
print("   -> Unido con dim_fecha.")

   -> Unido con dim_fecha.


In [11]:
df_modelo_estrella.head(3)

,transaction_id,fecha,año,mes,cliente_id,producto_id,canal_id,sucursal_id,cantidad,precio_unitario_ars_nominal,monto_venta_ars_nominal,monto_venta_ars_real_2018,es_evento_x,anio_origen,dia,nombre_mes,nombre_dia_semana,trimestre,semana_del_año,es_evento_y
0,1,2018-01-01,2018,1,5739,3,2,2,3,7944.48,23833.45,23833.45,0,2018,1,January,Monday,1,1,0
1,2,2018-01-01,2018,1,11442,22,2,15,2,3984.98,7969.96,7969.96,0,2018,1,January,Monday,1,1,0
2,3,2018-01-01,2018,1,9441,11,2,7,1,19582.89,19582.89,19582.89,0,2018,1,January,Monday,1,1,0


### 4.2. Unir a Dimensión Productos

In [13]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_productos,
    on='producto_id',
    how='left'
)
print("   -> Unido con dim_productos.")

   -> Unido con dim_productos.


In [14]:
df_modelo_estrella.head(3)

,transaction_id,fecha,año,mes,cliente_id,producto_id,canal_id,sucursal_id,cantidad,precio_unitario_ars_nominal,...,dia,nombre_mes,nombre_dia_semana,trimestre,semana_del_año,es_evento_y,nombre_producto,categoria,subcategoria,precio_base_ars_2018
0,1,2018-01-01,2018,1,5739,3,2,2,3,7944.48,...,1,January,Monday,1,1,0,"Monitor 27""",Tecnologia,Computo,8000.0
1,2,2018-01-01,2018,1,11442,22,2,15,2,3984.98,...,1,January,Monday,1,1,0,Calefactor,Linea Blanca,Climatizacion,4000.0
2,3,2018-01-01,2018,1,9441,11,2,7,1,19582.89,...,1,January,Monday,1,1,0,"Smart TV 55""",Tecnologia,TV & Entretenimiento,20000.0


### 4.3. Unir a Dimensión Clientes

In [15]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_clientes,
    on='cliente_id',
    how='left'
)
print("   -> Unido con dim_clientes.")

   -> Unido con dim_clientes.


In [16]:
df_modelo_estrella.head()

,transaction_id,fecha,año,mes,cliente_id,producto_id,canal_id,sucursal_id,cantidad,precio_unitario_ars_nominal,...,semana_del_año,es_evento_y,nombre_producto,categoria,subcategoria,precio_base_ars_2018,nombre_cliente,email_cliente,ciudad_cliente,segmento_base
0,1,2018-01-01,2018,1,5739,3,2,2,3,7944.48,...,1,0,"Monitor 27""",Tecnologia,Computo,8000.0,Tomàs Benjamin Campos Muñoz,uflores@example.com,Paraná,Premium
1,2,2018-01-01,2018,1,11442,22,2,15,2,3984.98,...,1,0,Calefactor,Linea Blanca,Climatizacion,4000.0,Benjamin Alejandro Thiago Daniel Martinez,ianmansilla@example.com,San Luis,Frecuente
2,3,2018-01-01,2018,1,9441,11,2,7,1,19582.89,...,1,0,"Smart TV 55""",Tecnologia,TV & Entretenimiento,20000.0,Sebastian Chavez Sosa,alfonsina48@example.org,La Rioja,Frecuente
3,4,2018-01-01,2018,1,9808,24,2,25,3,2957.86,...,1,0,Licuadora,Pequeños Electro,Cocina & Hogar,3000.0,Lautaro Cordoba Rojas,juan-pablo70@example.com,Posadas,Ocasional
4,5,2018-01-01,2018,1,13384,22,2,13,2,3927.82,...,1,0,Calefactor,Linea Blanca,Climatizacion,4000.0,Dr(a). Sofia Ramos,vegaisabella@example.com,Chilecito,Frecuente


### 4.4. Unir a Dimensión Canales

In [17]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_canales,
    on='canal_id',
    how='left'
)
print("   -> Unido con dim_canales.")

   -> Unido con dim_canales.


In [18]:
df_modelo_estrella.head(3)

,transaction_id,fecha,año,mes,cliente_id,producto_id,canal_id,sucursal_id,cantidad,precio_unitario_ars_nominal,...,es_evento_y,nombre_producto,categoria,subcategoria,precio_base_ars_2018,nombre_cliente,email_cliente,ciudad_cliente,segmento_base,nombre_canal
0,1,2018-01-01,2018,1,5739,3,2,2,3,7944.48,...,0,"Monitor 27""",Tecnologia,Computo,8000.0,Tomàs Benjamin Campos Muñoz,uflores@example.com,Paraná,Premium,Sucursal Fisica
1,2,2018-01-01,2018,1,11442,22,2,15,2,3984.98,...,0,Calefactor,Linea Blanca,Climatizacion,4000.0,Benjamin Alejandro Thiago Daniel Martinez,ianmansilla@example.com,San Luis,Frecuente,Sucursal Fisica
2,3,2018-01-01,2018,1,9441,11,2,7,1,19582.89,...,0,"Smart TV 55""",Tecnologia,TV & Entretenimiento,20000.0,Sebastian Chavez Sosa,alfonsina48@example.org,La Rioja,Frecuente,Sucursal Fisica


### 4.5. Unir a Dimensión Sucursales

In [19]:
df_modelo_estrella = pd.merge(
    df_modelo_estrella,
    df_sucursales,
    on="sucursal_id",
    how="left"
)

print("   -> Unido con dim_sucursales incluyendo el canal Online.")

   -> Unido con dim_sucursales incluyendo el canal Online.


In [20]:
df_modelo_estrella.head(3)

,transaction_id,fecha,año,mes,cliente_id,producto_id,canal_id,sucursal_id,cantidad,precio_unitario_ars_nominal,...,subcategoria,precio_base_ars_2018,nombre_cliente,email_cliente,ciudad_cliente,segmento_base,nombre_canal,nombre_sucursal,direccion_sucursal,provincia_sucursal
0,1,2018-01-01,2018,1,5739,3,2,2,3,7944.48,...,Computo,8000.0,Tomàs Benjamin Campos Muñoz,uflores@example.com,Paraná,Premium,Sucursal Fisica,Sucursal 2,"Av. Santa Rosa N° 9560 Torre 2 Dto. 1, Viedma ...",Buenos Aires
1,2,2018-01-01,2018,1,11442,22,2,15,2,3984.98,...,Climatizacion,4000.0,Benjamin Alejandro Thiago Daniel Martinez,ianmansilla@example.com,San Luis,Frecuente,Sucursal Fisica,Sucursal 15,"Av. Posadas N° 435, Viedma 8500, Río Negro",Río Negro
2,3,2018-01-01,2018,1,9441,11,2,7,1,19582.89,...,TV & Entretenimiento,20000.0,Sebastian Chavez Sosa,alfonsina48@example.org,La Rioja,Frecuente,Sucursal Fisica,Sucursal 7,"Avenida 1 N° 85, Mendoza 5500, Mendoza",Chaco


## 5. Selección y Limpieza Final

In [21]:
# Renombrar columnas clave para la historia de datos
df_modelo_estrella.rename(columns={
    'monto_venta_usd': 'Monto_Venta_USD',
    'nombre_canal': 'Canal_Venta',
    'provincia_sucursal': 'Provincia'
}, inplace=True)

# Eliminar IDs intermedios si ya no son necesarios (o mantenerlos para drill-down)
df_modelo_estrella = df_modelo_estrella.drop(columns=['cliente_id', 'producto_id', 'canal_id', 'sucursal_id', 'fecha', 'dia', 'mes', 'año'], errors='ignore')

# Mostrar las primeras filas y columnas clave para validar
print("\n✔️ ¡Modelo Estrella consolidado finalizado!")
print(f"Número total de transacciones: {len(df_modelo_estrella):,}")
print("Columnas clave del DataFrame final:")
print(df_modelo_estrella.head(3).T)


✔️ ¡Modelo Estrella consolidado finalizado!
Número total de transacciones: 1,211,170
Columnas clave del DataFrame final:
                                                                             0  \
transaction_id                                                               1   
cantidad                                                                     3   
precio_unitario_ars_nominal                                            7944.48   
monto_venta_ars_nominal                                               23833.45   
monto_venta_ars_real_2018                                             23833.45   
es_evento_x                                                                  0   
anio_origen                                                               2018   
nombre_mes                                                             January   
nombre_dia_semana                                                       Monday   
trimestre                                                 

## 6. Exportar el DataFrame Consolidado

In [22]:
# Puedes exportar el modelo estrella final de nuevo a Parquet o a un formato más simple
FINAL_OUTPUT_PATH = 'data/analytical_layer/tecnoStore_modelo_estrella.parquet'
os.makedirs(os.path.dirname(FINAL_OUTPUT_PATH), exist_ok=True)
df_modelo_estrella.to_parquet(FINAL_OUTPUT_PATH, index=False)
print(f"\n💾 DataFrame final exportado a: {FINAL_OUTPUT_PATH}")


💾 DataFrame final exportado a: data/analytical_layer/tecnoStore_modelo_estrella.parquet
